In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
adict = {"Item Title":[], "Item Price":[]}
#ebay page url, for loop adds page number
ebayUrl = "https://www.ebay.co.uk/sch/i.html?_from=R40&_nkw=20kg+weight+plates&_in_kw=1&_ex_kw=&_sacat=0&LH_Sold=1&_udlo=&_udhi=&_samilow=&_samihi=&_sadis=15&_sargn=-1%26saslc%3D1&_fsradio2=%26LH_LocatedIn%3D1&_salic=3&LH_SubLocation=1&_sop=13&_dmd=1&_ipg=60&LH_Complete=&_pgn="

In [3]:
pageUrl = ebayUrl+str(5)
r= requests.get(pageUrl)
data=r.text
soup=BeautifulSoup(data, 'lxml')

In [7]:
listings = soup.find_all(class_="s-item s-item__pl-on-bottom")

In [8]:
#for each in listings:
#    for line in each.find_all("li"):
#        print(line.find(class_='s-item__wrapper clearfix'))

for each in listings:
    print(each.find(class_='s-item__title'))

In [11]:
for item in soup.select('.s-item__wrapper.clearfix'):
    print(item)

In [13]:
listings = soup.find_all('li', attrs={'class': 's-item'})
print(listings)

[]


In [14]:
from lxml import html

In [15]:
parser = html.fromstring(r.text)

In [16]:
product_listings = parser.xpath('//li[contains(@id,"results-listing")]')
raw_result_count = parser.xpath("//h1[contains(@class,'count-heading')]//text()")

In [17]:
raw_result_count

[]

In [18]:
product_listings

[]

In [9]:
print(pd.read_csv('~/code/non_finance/weightPlateDataTrial.csv'))

     Unnamed: 0                                         Item Title Item Price  \
0             0  ELEIKO SPORT WEIGHTLIFTING TRAINING DISC - BLA...    £180.00   
1             1  10 / 2.5 / 1.25KG Weight Plates Set for Gym Ba...      £2.20   
2             2  Premium Colour Bumper Plates | Steel insert | ...     £44.99   
3             3  Olympic Rubber Weight Plate 10kg 15kg 20kg 25k...     £24.99   
4             4                 Crane Weight plates bundle 8x 1 kg     £15.00   
..          ...                                                ...        ...   
177         172   2 x 20kg Olympic Weights. 2 Sets Of 2 Available.     £70.00   
178         173  150KG 2" Olympic Bumper Weight Plate Set - 5kg...    £280.00   
179         174  10kg/20kg Dumbells Pair of Gym Weights Barbell...     £13.59   
180         175  10KG x 2 Cast Iron Barbell 25mm Weight Plates ...     £45.00   
181         176  2 x 10KG Cast Iron Barbell Dumbbell Weight Pla...     £45.00   

     Unnamed: 0.1  
0      

In [5]:
adict = {"Item Title":[], "Item Price":[]}

In [7]:
adict

{'Item Title': [], 'Item Price': []}